In [93]:
import re
import pandas as pd
import numpy as np
import math
import collections # 词频统计库
from keras.models import Sequential 
from keras.layers.embeddings import Embedding 
from keras.preprocessing import sequence 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
#计算TF
def TF(corpus,word_all):
    weight_tf = [[] for i in corpus]
    for word in word_all:
        for i in range(len(corpus)):
            temp_list = corpus[i].split()
            n1 = temp_list.count(word)
            n2 = len(temp_list)
            tf = n1/n2
            weight_tf[i].append(tf)
    return weight_tf

#计算二范式归一化的weight
def L2(weight):
    l2_weight = [[] for i in range(len(weight))]
    for e in range(len(weight)):
        sum = 0
        for env in weight[e]:
            sum += env**2
        sum = sum**0.5
        for env in weight[e]:
            l2_weight[e].append(env/sum)
    return l2_weight

#计算TF、IDF、TF-IDF并返回
def TF_IDF(corpus, word_all):
    weight = [[] for i in corpus]
    weight_tf = [[] for i in corpus]
    n3 = len(corpus)
    for word in word_all:
        print(word)
        #for i in range(len(corpus)):
        for i in range(100):
            #print(i)
            temp_list = corpus[i].split()
            n1 = temp_list.count(word)
            n2 = len(temp_list)
            tf = n1 / n2
            n4 = 0
            weight_tf[i].append(tf)
            for eve in corpus:
                temp_list_ = eve.split()
                if word in temp_list_:
                    n4 += 1
            idf = math.log( ((n3+1)/(n4+1)),10)
            weight[i].append(tf*idf)
        
    l2_weight = L2(weight)
    return weight_tf, l2_weight



In [243]:
#读取数据
with open("C:\\Users\\azhi\\Desktop\\作业\\自然语言处理\\wordLib3.txt","r",encoding='utf-8') as f:
    file = f.read()
word_all = file.split()
with open("C:\\Users\\azhi\\Desktop\\作业\\自然语言处理\\review3.txt","r",encoding='utf-8') as F:
    review = F.readlines()
f.close()
F.close()

In [244]:
sentence = []
for i in range(len(review)):
    w = review[i].split()
    sentence.append(w)


In [249]:
model=gensim.models.Word2Vec(sentence,sg=1,vector_size=100,window=5,min_count=2,negative=3,sample=0.001,hs=1,workers=4)

In [250]:
model.wv.save_word2vec_format("C:\\Users\\azhi\\Desktop\\作业\\自然语言处理\\word2vec3.bin",binary = "Ture") 

In [251]:
#model.wv.most_similar('touch')  

In [253]:
tf = TF(review,word_all)

In [254]:
#将tf写入txt文件
f=open("C:\\Users\\azhi\\Desktop\\作业\\自然语言处理\\tf3.txt","w",encoding='utf-8')
for vec in tf:
    for num in vec:
        f.write(str(num) + ' ')
    f.write('\n')
f.close()

In [30]:
data = pd.read_csv('C:\\Users\\azhi\\Desktop\\作业\\自然语言处理\\IMDB Dataset.csv')
Y = data["sentiment"]
Y = np.array(Y)
Y = Y.tolist()
for i in range(len(Y)):
    if Y[i] == 'positive':
        Y[i] = 1
    else:
        Y[i] = 0

In [24]:
model = Sequential()
#model.add(Embedding(2659, 64, input_length = 20000))
model.add(Conv1D(filters = 64, kernel_size = 3,input_shape=(2659, 1), padding = 'same', activation = 'relu'))
model.add(MaxPooling1D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Conv1D(filters = 128, kernel_size = 3, padding = 'same',activation = 'relu'))
model.add(MaxPooling1D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 2659, 64)          256       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 1329, 64)         0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 1329, 64)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 1329, 128)         24704     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 664, 128)         0         
 1D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 664, 128)         

In [252]:
tf_train = tf[0:30000]
tf_valid = tf[30000:40000]
tf_test = tf[40000:]
y_train = Y[0:30000]
y_valid = Y[30000:40000]
y_test = Y[40000:]

In [42]:
# 训练
history = model.fit(tf_train, y_train, epochs=20, batch_size=100,
                    verbose=1, validation_data=[tf_valid, y_valid])
  

Epoch 1/20
301/301 [==============================] - 191s 634ms/step - loss: 0.4125 - accuracy: 0.8205 - val_loss: 0.4835 - val_accuracy: 0.7528
Epoch 2/20
301/301 [==============================] - 125s 415ms/step - loss: 0.3619 - accuracy: 0.8428 - val_loss: 0.4013 - val_accuracy: 0.8277
Epoch 3/20
301/301 [==============================] - 124s 413ms/step - loss: 0.3269 - accuracy: 0.8625 - val_loss: 0.4211 - val_accuracy: 0.8365
Epoch 4/20
301/301 [==============================] - 124s 412ms/step - loss: 0.2900 - accuracy: 0.8811 - val_loss: 0.3948 - val_accuracy: 0.8379
Epoch 5/20
301/301 [==============================] - 122s 407ms/step - loss: 0.2536 - accuracy: 0.8974 - val_loss: 0.4016 - val_accuracy: 0.8329
Epoch 6/20
301/301 [==============================] - 124s 413ms/step - loss: 0.2144 - accuracy: 0.9164 - val_loss: 0.4326 - val_accuracy: 0.8317
Epoch 7/20
301/301 [==============================] - 124s 411ms/step - loss: 0.1720 - accuracy: 0.9348 - val_loss: 0.4739 -

In [43]:
model_file = 'C:\\Users\\azhi\\Desktop\\作业\\自然语言处理\\cnn_model.h5'
model.save(model_file)

In [23]:
input_array = np.random.randint(10, size=(32, 10))
print(type(input_array))

<class 'numpy.ndarray'>


In [44]:
tf_pred = model.predict(tf_test)

In [37]:
print(len(tf_pred))
print(len(y_test))

9999
9999


In [50]:
for i in range(len(tf_pred)):
    if(tf_pred[i] > 0.5):
        tf_pred[i] = 1
    else:
        tf_pred[i] = 0

In [70]:
 #计算精度、查准率P、查全率R、

def cal_acc(pred,y_test):
    acc = 0;
    for i in range(len(pred)):
        acc += abs(pred[i]-y_test[i])
    acc /= len(pred)
    acc = 1-acc
    return acc

def cal_TP_FN_FP_TN(pred,y_test):
    TP = 0 
    FN = 0
    FP = 0
    TN = 0
    for i in range(len(pred)):
        if(pred[i] == 1 and y_test[i] == 1):
            TP += 1
        if(pred[i] == 1 and y_test[i] == 0):
            FP += 1
        if(pred[i] == 0 and y_test[i] == 0):
            TN += 1
        if(pred[i] == 0 and y_test[i] == 1):
            FN += 1
    return TP,FN,FP,TN

def cal_P(TP,FP):
    P = TP/(TP+FP)
    return P

def cal_R(TP,FN):
    R = TP / (TP+FN)
    return R

In [76]:
acc = cal_acc(tf_pred,y_test)
TP, FN, FP, TN = cal_TP_FN_FP_TN(tf_pred,y_test)
P = cal_P(TP,FP)
R = cal_R(TP,FN)
print("acc is "+str(acc[0]))
print("P is "+str(P))
print("R is "+str(R))

acc is 0.8189819
P is 0.8281667008827757
R is 0.8056720591172358


In [181]:
#MAX_SEQUENCE_LENGTH = 0
#for i in range(len(sentence)):
#    if(MAX_SEQUENCE_LENGTH < len(sentence[i])):
#        MAX_SEQUENCE_LENGTH = len(sentence[i])
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 300

In [245]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review)
sequences = tokenizer.texts_to_sequences(review)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
data = data.tolist()

Found 2659 unique tokens.


In [246]:
print(len(word_all))

2659


In [135]:
w2v_train = data[0:30000] 
w2v_valid = data[30000:40000]
w2v_test = data[40000:]

In [124]:
# 加载bin格式的模型
word2Vec = gensim.models.KeyedVectors.load_word2vec_format("word2Vec3.bin",binary=True)
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items(): 
    if str(word) in word2Vec:
        embedding_matrix[i] = np.asarray(word2Vec[str(word)],
                                         dtype='float32')


embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [136]:
#word2vec
w2vmodel = Sequential()
w2vmodel.add(embedding_layer)
#w2vmodel.add(Embedding(2659, 64, input_length = 20000))
w2vmodel.add(Conv1D(filters = 64, kernel_size = 3,input_shape=(2659, 1), padding = 'same', activation = 'relu'))
w2vmodel.add(MaxPooling1D(pool_size = 2))
w2vmodel.add(Dropout(0.25))
w2vmodel.add(Conv1D(filters = 128, kernel_size = 3, padding = 'same',activation = 'relu'))
w2vmodel.add(MaxPooling1D(pool_size = 2))
w2vmodel.add(Dropout(0.25))
w2vmodel.add(Flatten())
w2vmodel.add(Dense(64, activation = 'relu'))
w2vmodel.add(Dense(32, activation = 'relu'))
w2vmodel.add(Dense(1, activation = 'sigmoid'))
w2vmodel.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
print(w2vmodel.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 100)          1455300   
                                                                 
 conv1d_10 (Conv1D)          (None, 500, 64)           19264     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 250, 64)          0         
 g1D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 250, 64)           0         
                                                                 
 conv1d_11 (Conv1D)          (None, 250, 128)          24704     
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 125, 128)         0         
 g1D)                                                 

In [137]:
w2vmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
w2vmodel.fit(w2v_train, y_train, validation_data=(w2v_valid, y_valid), epochs=50, batch_size=128)
w2vmodel.save('word_vector_cnn.h5')

Epoch 1/50
235/235 [==============================] - 45s 189ms/step - loss: 0.0000e+00 - acc: 0.4998 - val_loss: 0.0000e+00 - val_acc: 0.5023
Epoch 2/50
235/235 [==============================] - 34s 147ms/step - loss: 0.0000e+00 - acc: 0.4995 - val_loss: 0.0000e+00 - val_acc: 0.5023
Epoch 3/50
235/235 [==============================] - 34s 144ms/step - loss: 0.0000e+00 - acc: 0.4995 - val_loss: 0.0000e+00 - val_acc: 0.5023
Epoch 4/50
235/235 [==============================] - 34s 145ms/step - loss: 0.0000e+00 - acc: 0.4995 - val_loss: 0.0000e+00 - val_acc: 0.5023
Epoch 5/50
235/235 [==============================] - 34s 144ms/step - loss: 0.0000e+00 - acc: 0.4995 - val_loss: 0.0000e+00 - val_acc: 0.5023
Epoch 6/50
235/235 [==============================] - 33s 142ms/step - loss: 0.0000e+00 - acc: 0.4995 - val_loss: 0.0000e+00 - val_acc: 0.5023
Epoch 7/50
235/235 [==============================] - 33s 142ms/step - loss: 0.0000e+00 - acc: 0.4995 - val_loss: 0.0000e+00 - val_acc: 0.5023

In [139]:
w2v_pred = w2vmodel.predict(w2v_test)
w2v_pred = w2v_pred.tolist()

In [154]:
w2v_list = []
for i in range(len(w2v_pred)):
    w2v_list.append(w2v_pred[i][0])

In [152]:
print(len(w2v_pred))

9999


In [155]:
w2vacc = cal_acc(w2v_list,y_test)
w2vTP, w2vFN, w2vFP, w2vTN = cal_TP_FN_FP_TN(w2v_list,y_test)
w2vP = cal_P(w2vTP,w2vFP)
w2vR = cal_R(w2vTP,w2vFN)
print("acc is "+str(w2vacc[0]))
print("P is "+str(w2vP))
print("R is "+str(w2vR))

ZeroDivisionError: division by zero